<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/Dacon%EC%8B%A0%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import plotly.graph_objects as go
import itertools

In [2]:
from pprint import pprint
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras

from kaggler.preprocessing import LabelEncoder
from kaggler.model import AutoLGB

In [3]:
import kaggler
print(kaggler.__version__)

0.8.13


In [70]:
pip install kaggler

     |████████████████████████████████| 819kB 8.0MB/s 
  Created wheel for kaggler: filename=Kaggler-0.8.13-cp37-cp37m-linux_x86_64.whl size=2137196 sha256=c9c9ba08a1d3c90dd071b0897998d6886abff97214d3dba248aea8d13b738701
  Stored in directory: /root/.cache/pip/wheels/ae/9f/ed/b41a4cb406be1bd951ec2898582aedebd3e2da341b06e61d8a
  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-cp37-none-any.whl size=7850 sha256=b292eed0705ec855579cfba82739c859006d4045f2ffbbd228047ccef245e54c
  Stored in directory: /root/.cache/pip/wheels/b3/61/2d/776be7b8a4f14c5db48c8e5451451cabc58dc6aa7ee3801163
Successfully built kaggler ml-metrics


In [4]:
train = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/신용카드 사용자 연체 예측 AI 경진대회/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/신용카드 사용자 연체 예측 AI 경진대회/test.csv')

In [5]:
submit = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/신용카드 사용자 연체 예측 AI 경진대회/sample_submission.csv')

In [14]:
def missing_data(data):
  total = data.isnull().sum()
  percent = round(data.isnull().sum()/len(data)*100, 2)
  table = pd.concat([total, percent], keys=['Total', 'Percent'], axis=1).sort_values(by='Percent', ascending=False)
  return table

In [15]:
missing_data(train)

,Total,Percent
occyp_type,8171,30.88
index,0,0.00
gender,0,0.00
begin_month,0,0.00
family_size,0,0.00
email,0,0.00
phone,0,0.00
work_phone,0,0.00
FLAG_MOBIL,0,0.00
DAYS_EMPLOYED,0,0.00


In [16]:
missing_data(test)

,Total,Percent
occyp_type,3152,31.52
index,0,0.00
DAYS_BIRTH,0,0.00
family_size,0,0.00
email,0,0.00
phone,0,0.00
work_phone,0,0.00
FLAG_MOBIL,0,0.00
DAYS_EMPLOYED,0,0.00
house_type,0,0.00


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          10000 non-null  int64  
 1   gender         10000 non-null  object 
 2   car            10000 non-null  object 
 3   reality        10000 non-null  object 
 4   child_num      10000 non-null  int64  
 5   income_total   10000 non-null  float64
 6   income_type    10000 non-null  object 
 7   edu_type       10000 non-null  object 
 8   family_type    10000 non-null  object 
 9   house_type     10000 non-null  object 
 10  DAYS_BIRTH     10000 non-null  int64  
 11  DAYS_EMPLOYED  10000 non-null  int64  
 12  FLAG_MOBIL     10000 non-null  int64  
 13  work_phone     10000 non-null  int64  
 14  phone          10000 non-null  int64  
 15  email          10000 non-null  int64  
 16  occyp_type     6848 non-null   object 
 17  family_size    10000 non-null  float64
 18  begin_m

In [19]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [6]:
# 나이
train['Age'] = np.round(train['DAYS_BIRTH'] / -365, 2)
test['Age'] = np.round(test['DAYS_BIRTH'] / -365, 2)
# 연차
train['Working_years'] = np.round(train['DAYS_EMPLOYED'] / -365, 2)
test['Working_years'] = np.round(test['DAYS_EMPLOYED'] / -365, 2)
# 빌린 연차
train['begin_years'] = np.round(train['begin_month']/ -12, 2)
test['begin_years'] = np.round(test['begin_month']/ -12, 2)

In [7]:
data = pd.concat([train.drop(['credit'], axis=1), test], axis=0)
data.shape

(36457, 22)

In [35]:
data.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,Age,Working_years,begin_years
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,38.08,12.90,0.50
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,31.18,4.22,0.42
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,52.29,12.15,1.83
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,41.34,5.73,3.08
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,41.20,5.77,2.17


In [36]:
missing_data(data)

,Total,Percent
occyp_type,11323,31.06
index,0,0.00
gender,0,0.00
Working_years,0,0.00
Age,0,0.00
begin_month,0,0.00
family_size,0,0.00
email,0,0.00
phone,0,0.00
work_phone,0,0.00


In [8]:
from sklearn.preprocessing import LabelEncoder

In [11]:
data_occyp_type = data.occyp_type

In [12]:
data = data.drop(['occyp_type'], axis=1)

In [13]:
## LabelEncoder
for i in data.select_dtypes('object').columns:
  encoder = LabelEncoder()
  data[i] = encoder.fit_transform(data[i])

In [14]:
data = pd.concat([data_occyp_type, data], axis=1)

In [15]:
## 데이터 나누기(occyp_type분류하기위한)
notnull = data[data['occyp_type'].notnull()]
notnull['occyp_type'] = encoder.fit_transform(notnull['occyp_type'])
notnull_x = notnull.drop(['occyp_type'], axis=1)
notnull_y = notnull['occyp_type']

isnull = data[data['occyp_type'].isnull()]
isnull_x = isnull.drop(['occyp_type'], axis=1)
isnull_y = isnull['occyp_type']

In [16]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [17]:
model_rf = RandomForestClassifier(n_estimators=200)
model_rf.fit(notnull_x, notnull_y)
y_pred_isnull = model_rf.predict(isnull_x)
model_rf.score(notnull_x, notnull_y)

1.0

In [18]:
isnull['occyp_type'] = y_pred_isnull

In [19]:
data = pd.concat([notnull, isnull], axis=0)

In [20]:
data = data.sort_values(by='index')

In [14]:
y = train['credit']
n_trn = train.shape[0]
x = data.iloc[:n_trn]
x_test = data.iloc[n_trn:]
print(y.shape, x.shape, x_test.shape)

(26457,) (26457, 22) (10000, 22)


In [15]:
n_class = 3
n_fold = 5

params = {'num_class': n_class}

cv = StratifiedKFold(n_splits=n_fold)

p = np.zeros((x.shape[0], n_class), dtype=float)
p_tst = np.zeros((x_test.shape[0], n_class), dtype=float)
for i_cv, (i_trn, i_val) in enumerate(cv.split(x, y)):
    if i_cv == 0:
        clf = AutoLGB(objective='multiclass', metric='multi_logloss', params=params, 
                      feature_selection=False, n_est=10000)
        clf.tune(x.iloc[i_trn], y[i_trn])
        n_best = clf.n_best
        features = clf.features
        params = clf.params
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')        
        pprint(params)
        clf.fit(x.iloc[i_trn], y[i_trn])
    else:
        train_data = lgb.Dataset(x[features].iloc[i_trn], label=y[i_trn])
        clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    p[i_val] = clf.predict(x[features].iloc[i_val])
    p_tst += clf.predict(x_test[features]) / n_fold

100%|██████████| 100/100 [03:26<00:00,  2.07s/it, best loss: 0.8004983791249084]
best iteration: 259
selected features (22): ['index', 'gender', 'car', 'reality', 'child_num', 'income_total', 'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'occyp_type', 'family_size', 'begin_month', 'Age', 'Working_years', 'begin_years']
{'bagging_fraction': 0.9,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'feature_fraction': 0.5,
 'feature_pre_filter': False,
 'lambda_l1': 0.1,
 'lambda_l2': 0,
 'learning_rate': 0.0154357783573234,
 'max_depth': -1,
 'metric': 'multi_logloss',
 'min_child_samples': 10,
 'num_class': 3,
 'num_leaves': 127,
 'num_threads': -1,
 'objective': 'multiclass',
 'seed': 42,
 'verbosity': -1}


In [16]:
print(f'CV Log Loss: {log_loss(y, p):.6f}')

CV Log Loss: 0.874601


In [ ]:
submit[submit.columns] = p_tst
submit.head()

In [ ]:
submit.to_csv(submission_file)